In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import acnportal
import sklearn

from copy import deepcopy
import warnings
import pytz
import numpy as np
import pandas as pd
import pickle
from datetime import datetime
from enum import Enum
from collections import namedtuple

from acnportal import acnsim
from acnportal import algorithms
from acnportal.acnsim.events import EventQueue
from acnportal.signals.tariffs.tou_tariff import TimeOfUseTariff
from acnportal.acnsim.events import GaussianMixtureEvents
from acnportal.contrib.acnsim import StochasticNetwork
from acnportal.acnsim.network import ChargingNetwork
from modified_adacharge import modified_adacharge
from modified_simulator import modified_simulator

In [3]:
def ev_fleet_level_2_network(transformer_cap=30, evse_per_phase=2):
    """ Configurable charging network for level-2 EVSEs connected line to line
        at 415 V.

    Args:
        transformer_cap (float): Capacity of the transformer feeding the network
          [kW]
        evse_per_phase (int): Number of EVSEs on each phase. Total number of
          EVSEs will be 3 * evse_per_phase.

    Returns:
        ChargingNetwork: Configured ChargingNetwork.
    """
    network = StochasticNetwork(early_departure=True)
    # network = ChargingNetwork()
    voltage = 415
    # evse_type = 'AeroVironment'
    evse_type = 'ClipperCreek'

    # Define the sets of EVSEs in the Caltech ACN.
    AB_ids = ['AB-{0}'.format(i) for i in range(evse_per_phase)]
    BC_ids = ['BC-{0}'.format(i) for i in range(evse_per_phase)]
    CA_ids = ['CA-{0}'.format(i) for i in range(evse_per_phase)]

    # print(AB_ids)

    # Add Caltech EVSEs
    for evse_id in AB_ids:
        network.register_evse(acnsim.get_evse_by_type(evse_id, evse_type), voltage, 30)
    for evse_id in BC_ids:
        network.register_evse(acnsim.get_evse_by_type(evse_id, evse_type), voltage, -90)
    for evse_id in CA_ids:
        network.register_evse(acnsim.get_evse_by_type(evse_id, evse_type), voltage, 150)

    # Add Caltech Constraint Set
    AB = acnsim.Current(AB_ids)
    BC = acnsim.Current(BC_ids)
    CA = acnsim.Current(CA_ids)

    # Define intermediate currents
    I3a = AB - CA
    I3b = BC - AB
    I3c = CA - BC
    I2a = (1 / 4) * (I3a - I3c)
    I2b = (1 / 4) * (I3b - I3a)
    I2c = (1 / 4) * (I3c - I3b)

    # Build constraint set
    primary_side_constr = transformer_cap * 1000 / 3 / 415
    print(primary_side_constr)
    secondary_side_constr = transformer_cap * 1000 / 3 / 230
    print(secondary_side_constr)
    network.add_constraint(I3a, secondary_side_constr, name='Secondary A')
    network.add_constraint(I3b, secondary_side_constr, name='Secondary B')
    network.add_constraint(I3c, secondary_side_constr, name='Secondary C')
    network.add_constraint(I2a, primary_side_constr, name='Primary A')
    network.add_constraint(I2b, primary_side_constr, name='Primary B')
    network.add_constraint(I2c, primary_side_constr, name='Primary C')

    return network

In [4]:
# How long each time discrete time interval in the simulation should be.
PERIOD = 5  # minutes

# Voltage of the network.
VOLTAGE = 415  # volts

# Default maximum charging rate for each EV battery.
DEFAULT_BATTERY_POWER = 11 # kW

In [20]:
level2_ev_fleet_network = ev_fleet_level_2_network(transformer_cap=30, evse_per_phase=2)
level2_ev_fleet_network.magnitudes

24.096385542168676
43.47826086956522


array([43.47826087, 43.47826087, 43.47826087, 24.09638554, 24.09638554,
       24.09638554])

In [7]:
class CustomUnpicklerJPLdata(pickle.Unpickler):
    def find_class(self, module, name):
        if name == "sklearn.mixture.gaussian_mixture":
            return sklearn.mixture.GaussianMixture
        if name == "GaussianMixture":
            return sklearn.mixture.GaussianMixture
        return super().find_class(module, name)
    
def get_synth_events(sessions_per_day):
    gmm = CustomUnpicklerJPLdata(open('./data/jpl_weekday_40.pkl', "rb")).load()


    # Generate a list of the number of sessions to draw for each day.
    # This generates 30 days of charging demands.
    # num_evs = [0]*2 + [sessions_per_day]*5 + [0]*2 + [sessions_per_day]*5 + [0]*2 + \
    #           [sessions_per_day]*5 + [0]*2 + [sessions_per_day]*5 + [0]*2

    # Generate sessions for 1 day (weekdays only)
    num_evs = [0]*2 + [sessions_per_day]*1

    # Note that because we are drawing from a distribution, some sessions will be
    # invalid, we ignore these sessions and remove the corresponding plugin events.
    gen = GaussianMixtureEvents(pretrained_model=gmm, duration_min=0.08334)

    synth_events = gen.generate_events(num_evs, PERIOD, VOLTAGE, DEFAULT_BATTERY_POWER)
    return synth_events

In [8]:
# Events with 32 EVs per weekday
sessions_32 = get_synth_events(30)

c:\Users\s3955218\Anaconda3\envs\evsim\lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator GaussianMixture from version 0.20.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [8]:
Schedulingimportance = namedtuple("Schedulingimportance", "importancelevel")
# evSchedulingimportance = Schedulingimportance('high')
# print(evSchedulingimportance.level)

In [9]:
session_copy = deepcopy(sessions_32)
# print(type(session_copy))
session_list = list(session_copy.queue)
for session in session_list:
    print(session[0], session[1].ev._session_id, session[1].ev.departure, session[1].ev.requested_energy, session[1].event_type)


646 session_29 742 5.938367759865874 Plugin
652 session_2 771 8.334763696892274 Plugin
649 session_7 753 12.09071960470141 Plugin
663 session_18 750 2.852151524482916 Plugin
659 session_19 778 3.2038390153010723 Plugin
662 session_1 809 0.9063720227375913 Plugin
661 session_12 756 5.5719021125395365 Plugin
668 session_15 795 18.0150619903883 Plugin
666 session_0 785 0.7302074561875822 Plugin
669 session_20 788 8.08405461376015 Plugin
662 session_21 770 12.000161465139612 Plugin
663 session_24 712 22.438822517464832 Plugin
666 session_26 779 2.8179727334302265 Plugin
675 session_6 759 20.10523365817337 Plugin
663 session_14 776 18.572208101827844 Plugin
706 session_3 780 5.052721216374088 Plugin
734 session_16 794 8.20743637469259 Plugin
775 session_17 792 5.594200742103285 Plugin
667 session_8 713 13.14549137828789 Plugin
732 session_4 777 13.051242213697352 Plugin
670 session_9 707 12.336282721139847 Plugin
742 session_10 745 0.5 Plugin
671 session_22 777 7.547502848620931 Plugin
736 

In [28]:
# This function tags a specific session with a high importance level. 
# To accomplish this, it creates a new session list with the desired session tagged as high and the rest as low.
# It changes event_queue module that is located in acnsim package. (C:\Users\s3955218\Anaconda3\envs\evsim\Lib\site-packages\acnportal\acnsim\events\event_queue.py)
# In the event_queue module, it changes the EventQueue class (add_event function)
# TO DO: Modify to take multiple sessions as input and tag them as high
def tag_specific_session_for_scheduling(session_list, session_id: str) -> list:
    modified_session_list = []
    for session in session_list:
        if session[1].ev._session_id == session_id:
            modified_session = (session[1], 'high')
            modified_session_list.append(modified_session)
        else:
            modified_session_list.append(session[1])
    return modified_session_list

new_session_list = tag_specific_session_for_scheduling(session_list, 'session_25')
new_session_32 = EventQueue(new_session_list)

new_session_copy = deepcopy(new_session_32)
new_modified_session_list = list(new_session_copy.queue)

for sess in new_modified_session_list:
    if len(sess) == 3:
        print("printing tuple event")
        print(sess)
        print(sess[0], sess[1].ev._session_id, sess[1].ev.departure, sess[1].ev.requested_energy, sess[1].event_type, sess[2])
    else:
        print(sess)
        print(sess[0], sess[1].ev._session_id, sess[1].ev.departure, sess[1].ev.requested_energy, sess[1].event_type)
    # print(session[0], session[1].ev._session_id, session[1].ev.departure, session[1].ev.requested_energy, session[1].event_type, session[2].importancelevel)

(587, acnportal.acnsim.events.event.PluginEvent(timestamp=587, event_type=<str object at 0x000002661CA592B0>, precedence=10, ev=<acnportal.acnsim.models.ev.EV object at 0x000002665A3EB250>))
587 session_29 654 14.025545202883222 Plugin
(644, acnportal.acnsim.events.event.PluginEvent(timestamp=644, event_type=<str object at 0x000002661CA592B0>, precedence=10, ev=<acnportal.acnsim.models.ev.EV object at 0x0000026655D28370>))
644 session_31 783 5.888277680592424 Plugin
(647, acnportal.acnsim.events.event.PluginEvent(timestamp=647, event_type=<str object at 0x000002661CA592B0>, precedence=10, ev=<acnportal.acnsim.models.ev.EV object at 0x0000026655D28850>))
647 session_20 760 12.450528194396789 Plugin
(647, acnportal.acnsim.events.event.PluginEvent(timestamp=647, event_type=<str object at 0x000002661CA592B0>, precedence=10, ev=<acnportal.acnsim.models.ev.EV object at 0x00000266582A3EB0>))
647 session_21 757 11.907357770730288 Plugin
(651, acnportal.acnsim.events.event.PluginEvent(timestamp

In [10]:
def run_experiment(network, algorithm, events):
    """ Run simulation for the events defined previously and the specified
        network / algorithm / events.
    """
    # Timezone of the ACN we are using.
    # timezone = pytz.timezone('America/Los_Angeles')
    # Australian timezone for the experiment
    timezone = pytz.timezone('Australia/Melbourne')

    # Start and End times are used when collecting data.
    # start = timezone.localize(datetime(2019, 6, 1))
    # end = timezone.localize(datetime(2019, 7, 1))

    start = timezone.localize(datetime(2023, 10, 3))
    end = timezone.localize(datetime(2023, 10, 4))

    sch = deepcopy(algorithm)
    cn = deepcopy(network)
    signals = {'tariff': TimeOfUseTariff('sce_tou_ev_4_march_2019')}

    # sim = acnsim.Simulator(cn, sch, events, start, period=PERIOD, verbose=False, signals=signals)
    sim = modified_simulator.Simulator(cn, sch, events, start, period=PERIOD, verbose=False, signals=signals)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
    sim.run()

    r = {'proportion_of_energy_delivered': acnsim.proportion_of_energy_delivered(sim),
         'energy_delivered': sum(ev.energy_delivered for ev in sim.ev_history.values()),
         'num_swaps': cn.swaps,
         'num_never_charged': cn.never_charged,
         'energy_cost': acnsim.energy_cost(sim),
         'demand_charge': acnsim.demand_charge(sim)
         }
    
    # r['total_cost'] = r['energy_cost'] + r['demand_charge'] # original
    r['total_cost'] = r['energy_cost'] # modified
    r['$/kWh'] = r['total_cost'] / r['energy_delivered']
    # return r, sim.charging_rates_as_df()
    return r

In [11]:
uncontrolled = algorithms.UncontrolledCharging()
llf = algorithms.SortedSchedulingAlgo(algorithms.least_laxity_first)

In [12]:
cost_min_obj = [modified_adacharge.ObjectiveComponent(modified_adacharge.total_energy, 1000),
                modified_adacharge.ObjectiveComponent(modified_adacharge.tou_energy_cost),
                modified_adacharge.ObjectiveComponent(modified_adacharge.quick_charge, 1e-5),
                modified_adacharge.ObjectiveComponent(modified_adacharge.equal_share, 1e-12)
               ]
# peak limit: total aggregated current limit 
# cost_min = adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj, solver="MOSEK", quantize=True, reallocate=True, peak_limit=1000, max_recompute=1)
cost_min = modified_adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj, solver="MOSEK", quantize=True, reallocate=False, peak_limit=150, max_recompute=1)

#### Experimenting with Step 
- No need to run all the time. This part should only be run to understand internals of the code
- Understanding the behaviour of each algorithm standalone 

In [ ]:
 # Australian timezone for the experiment
timezone = pytz.timezone('Australia/Melbourne')

# Start and End times are used when collecting data.
# start = timezone.localize(datetime(2019, 6, 1))
# end = timezone.localize(datetime(2019, 7, 1))

start = timezone.localize(datetime(2023, 10, 3))
end = timezone.localize(datetime(2023, 10, 4))

sch = deepcopy(algorithms.SortedSchedulingAlgo(algorithms.least_laxity_first))
cn = deepcopy(level2_ev_fleet_network)
signals = {'tariff': TimeOfUseTariff('sce_tou_ev_4_march_2019')}

# sim = acnsim.Simulator(cn, sch, events, start, period=PERIOD, verbose=False, signals=signals)
sim = modified_simulator.Simulator(cn, sch, deepcopy(sessions_32), start, period=PERIOD, verbose=False, signals=signals)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
sim.run()

r = {'proportion_of_energy_delivered': acnsim.proportion_of_energy_delivered(sim),
        'energy_delivered': sum(ev.energy_delivered for ev in sim.ev_history.values()),
        'num_swaps': cn.swaps,
        'num_never_charged': cn.never_charged,
        'energy_cost': acnsim.energy_cost(sim),
        'demand_charge': acnsim.demand_charge(sim)
        }

# r['total_cost'] = r['energy_cost'] + r['demand_charge'] # original
r['total_cost'] = r['energy_cost'] # modified
r['$/kWh'] = r['total_cost'] / r['energy_delivered']

#### Run simulation from here

In [28]:
level2_30kW_untrl_32 = run_experiment(level2_ev_fleet_network, uncontrolled, deepcopy(sessions_32))

on-going session: ['session_28']
High priority EV charging sessions: []
on-going session: ['session_28', 'session_18']
High priority EV charging sessions: []
on-going session: ['session_28', 'session_18', 'session_2']
High priority EV charging sessions: []
on-going session: ['session_28', 'session_18', 'session_2', 'session_19']
High priority EV charging sessions: []
on-going session: ['session_28', 'session_2', 'session_19', 'session_29']
High priority EV charging sessions: []
on-going session: ['session_28', 'session_2', 'session_0', 'session_19', 'session_29']
High priority EV charging sessions: []
on-going session: ['session_2', 'session_0']
High priority EV charging sessions: []
on-going session: ['session_6', 'session_2', 'session_0']
High priority EV charging sessions: []
on-going session: ['session_6', 'session_2', 'session_0']
High priority EV charging sessions: []
on-going session: ['session_6', 'session_4', 'session_2', 'session_0', 'session_3']
High priority EV charging ses

c:\Users\s3955218\repos\acn-ev-simulation\modified_simulator\modified_simulator.py:355: UserWarning: Invalid schedule provided at iteration 639. Max violation is 20.511739130434783 A on Secondary A at time index 0.
  warnings.warn(
c:\Users\s3955218\repos\acn-ev-simulation\modified_simulator\modified_simulator.py:355: UserWarning: Invalid schedule provided at iteration 640. Max violation is 20.511739130434783 A on Secondary A at time index 0.
  warnings.warn(
c:\Users\s3955218\repos\acn-ev-simulation\modified_simulator\modified_simulator.py:355: UserWarning: Invalid schedule provided at iteration 641. Max violation is 20.511739130434783 A on Secondary A at time index 0.
  warnings.warn(
c:\Users\s3955218\repos\acn-ev-simulation\modified_simulator\modified_simulator.py:355: UserWarning: Invalid schedule provided at iteration 642. Max violation is 20.511739130434783 A on Secondary A at time index 0.
  warnings.warn(
c:\Users\s3955218\repos\acn-ev-simulation\modified_simulator\modified_si

In [ ]:
# Run simulation with modified sessions (tagged as high)
level2_30kW_untrl_32 = run_experiment(level2_ev_fleet_network, uncontrolled, deepcopy(new_session_32))

In [29]:
level2_30kW_llf_32 = run_experiment(level2_ev_fleet_network, llf, deepcopy(sessions_32))

on-going session: ['session_28']
High priority EV charging sessions: []
on-going session: ['session_18', 'session_28']
High priority EV charging sessions: []
on-going session: ['session_18', 'session_2', 'session_28']
High priority EV charging sessions: []
on-going session: ['session_18', 'session_19', 'session_2', 'session_28']
High priority EV charging sessions: []
on-going session: ['session_18', 'session_19', 'session_29', 'session_2', 'session_28']
High priority EV charging sessions: []
on-going session: ['session_18', 'session_19', 'session_29', 'session_2', 'session_28', 'session_0']
High priority EV charging sessions: []
on-going session: ['session_18', 'session_19', 'session_29', 'session_2', 'session_28', 'session_0']
High priority EV charging sessions: []
on-going session: ['session_18', 'session_19', 'session_29', 'session_2', 'session_28', 'session_0']
High priority EV charging sessions: []
on-going session: ['session_18', 'session_19', 'session_29', 'session_2', 'session_

In [ ]:
# Run simulation with modified sessions (tagged as high)
level2_30kW_llf_32 = run_experiment(level2_ev_fleet_network, llf, deepcopy(new_session_32))

In [30]:
level2_30kW_cost_min_32 = run_experiment(level2_ev_fleet_network, cost_min, deepcopy(sessions_32))

on-going session: ['session_28']
High priority EV charging sessions: []
on-going session: ['session_18', 'session_28']
High priority EV charging sessions: []
on-going session: ['session_2', 'session_18', 'session_28']
High priority EV charging sessions: []
on-going session: ['session_2', 'session_18', 'session_28', 'session_19']
High priority EV charging sessions: []
on-going session: ['session_2', 'session_18', 'session_28', 'session_29', 'session_19']
High priority EV charging sessions: []
on-going session: ['session_2', 'session_18', 'session_28', 'session_0', 'session_29', 'session_19']
High priority EV charging sessions: []
on-going session: ['session_2', 'session_18', 'session_28', 'session_0', 'session_29', 'session_19']
High priority EV charging sessions: []
on-going session: ['session_2', 'session_18', 'session_28', 'session_0', 'session_29', 'session_19']
High priority EV charging sessions: []
on-going session: ['session_2', 'session_18', 'session_28', 'session_0', 'session_2

In [ ]:
# Run simulation with modified sessions (tagged as high)
level2_30kW_cost_min_32 = run_experiment(level2_ev_fleet_network, cost_min, deepcopy(new_session_32))

In [31]:
ev_32 = pd.DataFrame({
    'Level 2: Unctrl: 30 kW : 6 EVSEs':  level2_30kW_untrl_32,
    'Level 2: LLF: 30 kW : 6 EVSEs': level2_30kW_llf_32,
    'Level 2: Min Cost: 30 kW : 6 EVSEs': level2_30kW_cost_min_32
})

In [32]:
ev_32

,Level 2: Unctrl: 30 kW : 6 EVSEs,Level 2: LLF: 30 kW : 6 EVSEs,Level 2: Min Cost: 30 kW : 6 EVSEs
proportion_of_energy_delivered,0.999998,0.449176,0.489730
energy_delivered,347.517138,156.096667,170.190000
num_swaps,21.000000,14.000000,15.000000
num_never_charged,0.000000,10.000000,9.000000
energy_cost,24.774602,11.692487,13.019097
demand_charge,1023.660000,463.438800,411.945600
total_cost,24.774602,11.692487,13.019097
$/kWh,0.071290,0.074905,0.076497


In [15]:
ev_32.to_csv("results/32_EV_simulation.csv")

In [16]:
pd.read_csv("results/32_EV_simulation.csv", index_col=0)

,Level 2: Unctrl: 30 kW : 6 EVSEs,Level 2: LLF: 30 kW : 6 EVSEs,Level 2: Min Cost: 30 kW : 6 EVSEs
proportion_of_energy_delivered,1.000000,0.422235,0.446793
energy_delivered,399.089403,168.509583,178.310417
num_swaps,22.000000,13.000000,13.000000
num_never_charged,0.000000,13.000000,13.000000
energy_cost,22.440797,9.475294,10.026395
demand_charge,856.152000,361.538100,458.087850
total_cost,878.592797,371.013394,468.114245
$/kWh,2.201494,2.201735,2.625277


In [19]:
level2_35kw_ev_fleet_network = ev_fleet_level_2_network(transformer_cap=35, evse_per_phase=2)
level2_35kw_ev_fleet_network.magnitudes

28.112449799196785
50.724637681159415


array([50.72463768, 50.72463768, 50.72463768, 28.1124498 , 28.1124498 ,
       28.1124498 ])

In [ ]:
level2_35kW_untrl_32 = run_experiment(level2_35kw_ev_fleet_network, uncontrolled, deepcopy(sessions_32))

In [22]:
level2_35kW_llf_32 = run_experiment(level2_35kw_ev_fleet_network, llf, deepcopy(sessions_32))

Unplugged EV session_20 from station CA-1 before full charge
Will plug in priority EV session_13
Unplugged EV session_18 from station BC-1 before full charge
Will plug in priority EV session_23
Unplugged EV session_30 from station CA-1 before full charge
Will plug in priority EV session_25
Unplugged EV session_29 from station BC-1 before full charge
Will plug in priority EV session_27


In [23]:
level2_35kW_cost_min_32 = run_experiment(level2_35kw_ev_fleet_network, cost_min, deepcopy(sessions_32))

Unplugged EV session_4 from station CA-0 before full charge
Will plug in priority EV session_13
Unplugged EV session_14 from station BC-1 before full charge
Will plug in priority EV session_28
Unplugged EV session_20 from station AB-0 before full charge
Will plug in priority EV session_23
Unplugged EV session_18 from station AB-1 before full charge
Will plug in priority EV session_25


In [24]:
ev_32_35kW = pd.DataFrame({
    'Level 2: Unctrl: 35 kW : 6 EVSEs':  level2_35kW_untrl_32,
    'Level 2: LLF: 35 kW : 6 EVSEs': level2_35kW_llf_32,
    'Level 2: Min Cost: 35 kW : 6 EVSEs': level2_35kW_llf_32
})

In [25]:
ev_32_35kW

,Level 2: Unctrl: 35 kW : 6 EVSEs,Level 2: LLF: 35 kW : 6 EVSEs,Level 2: Min Cost: 35 kW : 6 EVSEs
proportion_of_energy_delivered,0.997669,0.641357,0.641357
energy_delivered,354.425869,227.844583,227.844583
num_swaps,20.000000,14.000000,14.000000
num_never_charged,1.000000,12.000000,12.000000
energy_cost,25.508712,17.678608,17.678608
demand_charge,1023.660000,505.315800,505.315800
total_cost,25.508712,17.678608,17.678608
$/kWh,0.071972,0.077591,0.077591


In [24]:
# Events with 26 EVs per weekday
sessions_26 = get_synth_events(26)

c:\Users\s3955218\Anaconda3\envs\evsim\lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator GaussianMixture from version 0.20.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [25]:
uncontrolled_26 = algorithms.UncontrolledCharging()
llf_26 = algorithms.SortedSchedulingAlgo(algorithms.least_laxity_first)

In [26]:
cost_min_obj_26 = [
                modified_adacharge.ObjectiveComponent(modified_adacharge.total_energy, 1000),
                modified_adacharge.ObjectiveComponent(modified_adacharge.tou_energy_cost),
                modified_adacharge.ObjectiveComponent(modified_adacharge.quick_charge, 1e-5),
                modified_adacharge.ObjectiveComponent(modified_adacharge.equal_share, 1e-12)
               ]
# peak limit: total aggregated current limit 
# cost_min = adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj, solver="MOSEK", quantize=True, reallocate=True, peak_limit=1000, max_recompute=1)
cost_min_26 = modified_adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj_26, solver="MOSEK", quantize=True, reallocate=False, peak_limit=300, max_recompute=1)

In [27]:
level2_35kw_26_ev_fleet_network = ev_fleet_level_2_network(transformer_cap=35, evse_per_phase=2)

28.112449799196785
50.724637681159415


In [ ]:
level2_35kW_untrl_26 = run_experiment(level2_35kw_26_ev_fleet_network, uncontrolled, deepcopy(sessions_26))

In [29]:
level2_35kW_llf_26 = run_experiment(level2_35kw_26_ev_fleet_network, llf, deepcopy(sessions_26))

Unplugged EV session_18 from station AB-0 before full charge
Will plug in priority EV session_15
Unplugged EV session_8 from station CA-1 before full charge
Will plug in priority EV session_6
Unplugged EV session_4 from station AB-1 before full charge
Will plug in priority EV session_23
Unplugged EV session_17 from station AB-0 before full charge
Will plug in priority EV session_0
Unplugged EV session_6 from station CA-0 before full charge
Will plug in priority EV session_1
Unplugged EV session_5 from station AB-1 before full charge
Will plug in priority EV session_22
Unplugged EV session_24 from station AB-1 before full charge
Will plug in priority EV session_21
Unplugged EV session_22 from station BC-1 before full charge
Will plug in priority EV session_12


In [37]:
level2_35kW_cost_min_26 = run_experiment(level2_35kw_26_ev_fleet_network, cost_min_26, deepcopy(sessions_26))

Unplugged EV session_18 from station CA-0 before full charge
Will plug in priority EV session_14
Unplugged EV session_8 from station CA-1 before full charge
Will plug in priority EV session_24
Unplugged EV session_4 from station AB-1 before full charge
Will plug in priority EV session_23


In [38]:
ev_26_35kW = pd.DataFrame({
    'Level 2: Unctrl: 35 kW : 6 EVSEs':  level2_35kW_untrl_26,
    'Level 2: LLF: 35 kW : 6 EVSEs': level2_35kW_llf_26,
    'Level 2: Min Cost: 35 kW : 6 EVSEs': level2_35kW_cost_min_26
})

In [39]:
ev_26_35kW

,Level 2: Unctrl: 35 kW : 6 EVSEs,Level 2: LLF: 35 kW : 6 EVSEs,Level 2: Min Cost: 35 kW : 6 EVSEs
proportion_of_energy_delivered,1.000000,0.610762,0.484815
energy_delivered,304.478464,185.963750,147.615833
num_swaps,15.000000,12.000000,12.000000
num_never_charged,0.000000,5.000000,7.000000
energy_cost,23.138953,14.002298,10.573219
demand_charge,1023.660000,505.315800,511.830000
total_cost,23.138953,14.002298,10.573219
$/kWh,0.075995,0.075296,0.071627


In [40]:
cost_min_obj_26_2 = [
                modified_adacharge.ObjectiveComponent(modified_adacharge.total_energy, 1000),
                modified_adacharge.ObjectiveComponent(modified_adacharge.tou_energy_cost),
                modified_adacharge.ObjectiveComponent(modified_adacharge.quick_charge, 1e-5),
                modified_adacharge.ObjectiveComponent(modified_adacharge.equal_share, 1e-11)
               ]
# peak limit: total aggregated current limit 
# cost_min = adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj, solver="MOSEK", quantize=True, reallocate=True, peak_limit=1000, max_recompute=1)
cost_min_26_2 = modified_adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj_26_2, solver="MOSEK", quantize=True, reallocate=True, peak_limit=300, max_recompute=1)

In [41]:
level2_35kW_cost_min_26_2 = run_experiment(level2_35kw_26_ev_fleet_network, cost_min_26_2, deepcopy(sessions_26))

Unplugged EV session_18 from station AB-1 before full charge
Will plug in priority EV session_14
Unplugged EV session_4 from station AB-0 before full charge
Will plug in priority EV session_23
Unplugged EV session_6 from station AB-1 before full charge
Will plug in priority EV session_0


In [42]:
ev_26_35kW = pd.DataFrame({
    'Level 2: Unctrl: 35 kW : 6 EVSEs':  level2_35kW_untrl_26,
    'Level 2: LLF: 35 kW : 6 EVSEs': level2_35kW_llf_26,
    'Level 2: Min Cost: 35 kW : 6 EVSEs': level2_35kW_cost_min_26_2
})

In [44]:
ev_26_35kW

,Level 2: Unctrl: 35 kW : 6 EVSEs,Level 2: LLF: 35 kW : 6 EVSEs,Level 2: Min Cost: 35 kW : 6 EVSEs
proportion_of_energy_delivered,1.000000,0.610762,0.486013
energy_delivered,304.478464,185.963750,147.980590
num_swaps,15.000000,12.000000,12.000000
num_never_charged,0.000000,5.000000,7.000000
energy_cost,23.138953,14.002298,10.609457
demand_charge,1023.660000,505.315800,543.935700
total_cost,23.138953,14.002298,10.609457
$/kWh,0.075995,0.075296,0.071695


### Simulation (work in progress)
- Create a separate code module for MPC based Adaptive charging algorithm (done)
- Modify queue mechanism 
- Modify ada code to simulate fulfillment of charging requirement for high priority EVs
- Add new optimization objective 